# EDA Analysis on TTC Bus Delay Data

This data was sourced from open.toronto.ca website. Raw data can be found [here](https://open.toronto.ca/dataset/ttc-bus-delay-data/).

While there are multiple years of data available on this website, we are using 2024 data.

Our goal is to see how many buses are delayed and by how much time.

Let's get started with the EDA!

In [1]:
# This code block will be used for all library imports. 
import pandas as pd
import altair as alt

# Simplify working with large datasets in Altair
alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

## Loading and Fixing Data

In [2]:
# Let's load the data.
# the Dates column is parsed through date argument to make sure it is seen as datetime object.
ttc = pd.read_csv('data/ttc-bus-delay-data-2024.csv', parse_dates=['Date'])

/var/folders/sy/sq6z9ljj34g_lpl7p_k4z8kh0000gn/T/ipykernel_93791/317655958.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ttc = pd.read_csv('data/ttc-bus-delay-data-2024.csv', parse_dates=['Date'])


In [3]:
ttc.head()

,Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
0,2024-01-01,89,02:08,Monday,KEELE AND GLENLAKE,Vision,10,20,N,7107
1,2024-01-01,39,02:30,Monday,FINCH STATION,General Delay,20,40,NaN,8914
2,2024-01-01,300,03:13,Monday,BLOOR AND MANNING,General Delay,0,0,NaN,8562
3,2024-01-01,65,03:23,Monday,PARLIAMENT AND BLOOR,Security,0,0,N,8574
4,2024-01-01,113,03:37,Monday,MAIN STATION,Security,0,0,NaN,8541


In [4]:
# .shape shows up how big our data is
print(f"This dataset has {ttc.shape[0]} rows and {ttc.shape[1]} columns")
ttc.shape

This dataset has 45300 rows and 10 columns


(45300, 10)

In [5]:
# .info tells us about the object type for each column
# we can see 'Date' column was read as datetime object
ttc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45300 entries, 0 to 45299
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       45300 non-null  datetime64[ns]
 1   Route      44861 non-null  object        
 2   Time       45300 non-null  object        
 3   Day        45300 non-null  object        
 4   Location   45300 non-null  object        
 5   Incident   45300 non-null  object        
 6   Min Delay  45300 non-null  int64         
 7   Min Gap    45300 non-null  int64         
 8   Direction  38343 non-null  object        
 9   Vehicle    45300 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 3.5+ MB


In [6]:
# Let's also fix the 'Time' column
ttc1 = ttc.copy() # preserving the original

# Converting column to datetime object
ttc1['Time'] = pd.to_datetime(ttc['Time']).dt.time

/var/folders/sy/sq6z9ljj34g_lpl7p_k4z8kh0000gn/T/ipykernel_93791/4215578345.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ttc1['Time'] = pd.to_datetime(ttc['Time']).dt.time


Now, let's split 'Date' column into 'Date_' and 'Month' (year is not needed since this is for 2024), and convert 'Time' into 'Hours' so it more useful in the analysis later on.

In [7]:
ttc1['Date_'] = ttc1['Date'].dt.date
ttc1['Month'] = ttc1['Date'].dt.month
ttc1['Hour'] = ttc1['Time'].map(lambda x: x.hour) # used ChatGPT for this conversion

In [8]:
# Let's drop the 'Date' and 'Time' column since they are no longer needed
ttc1 = ttc1.drop(columns=['Date', 'Time'])

<br>

___
## Preprocessing
Now that the data is loaded in, we want to make sure we don't perform our analysis will null values. So, we will go through and identify any columns that have big number of null values and determine if they are worth keeping for analysis or not. 

In [9]:
ttc.isna().sum()

Date            0
Route         439
Time            0
Day             0
Location        0
Incident        0
Min Delay       0
Min Gap         0
Direction    6957
Vehicle         0
dtype: int64

In [10]:
# 'Direction' column has a lot null values. Let's see if these rows can be dropped
ttc[ttc['Direction'].isna()].head(20)

,Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
1,2024-01-01,39,02:30,Monday,FINCH STATION,General Delay,20,40,NaN,8914
2,2024-01-01,300,03:13,Monday,BLOOR AND MANNING,General Delay,0,0,NaN,8562
4,2024-01-01,113,03:37,Monday,MAIN STATION,Security,0,0,NaN,8541
12,2024-01-01,600,05:07,Monday,LAKESHORE AND THIRTY S,Cleaning - Unsanitary,25,50,NaN,8798
13,2024-01-01,300,05:38,Monday,KIPLING STATION,Emergency Services,0,0,NaN,3330
23,2024-01-01,41,07:21,Monday,KEELE AND IAN MACDONAL,Diversion,780,793,NaN,7251
24,2024-01-01,85,07:21,Monday,TORONTO ZOO,Security,20,40,NaN,3495
27,2024-01-01,NaN,08:08,Monday,EGLINTON DIVISION,Operations - Operator,0,0,NaN,0
28,2024-01-01,12,08:18,Monday,KENNEDY STATION,Emergency Services,30,60,NaN,8408
34,2024-01-01,900,09:21,Monday,KIPLING STATION,Operations - Operator,14,28,NaN,3341


<br>
Maybe deleting the rows isn't such a good idea because it makes up for a lot of overall data in the dataset. Instead, let's drop the column since we already have information about the route. For that reason, it is also okay to drop 'Vehicle' column as it is not needed for our goal of observing delays in busses.

In [11]:
ttc_clean = ttc.drop(columns=['Direction', 'Vehicle'])
ttc_clean.head()

,Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap
0,2024-01-01,89,02:08,Monday,KEELE AND GLENLAKE,Vision,10,20
1,2024-01-01,39,02:30,Monday,FINCH STATION,General Delay,20,40
2,2024-01-01,300,03:13,Monday,BLOOR AND MANNING,General Delay,0,0
3,2024-01-01,65,03:23,Monday,PARLIAMENT AND BLOOR,Security,0,0
4,2024-01-01,113,03:37,Monday,MAIN STATION,Security,0,0


Now, let's look at the Route column and its NaNs. This column is particularly important for us for our goal.

In [12]:
na_route = ttc[ttc['Route'].isna()]
na_route.head(10)

,Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
27,2024-01-01,NaN,08:08,Monday,EGLINTON DIVISION,Operations - Operator,0,0,NaN,0
69,2024-01-01,NaN,19:00,Monday,GUILDWOOD AND KINGSTON,Utilized Off Route,0,0,NaN,8491
213,2024-01-02,NaN,16:54,Tuesday,QUEENSWAY GARAGE,Mechanical,0,0,NaN,8136
248,2024-01-02,NaN,20:40,Tuesday,3RD FLOOR GUNN BUILDIN,Operations - Operator,0,0,NaN,0
293,2024-01-03,NaN,06:28,Wednesday,WILSON TRAINING OPERAT,Operations - Operator,0,0,NaN,0
350,2024-01-03,NaN,14:10,Wednesday,RUSSELL CARHOUSE,Mechanical,0,0,NaN,349
602,2024-01-04,NaN,17:57,Thursday,FINCH STATION,Operations - Operator,12,24,N,3253
624,2024-01-04,NaN,21:14,Thursday,ARROW GARAGE,Collision - TTC,0,0,NaN,1371
638,2024-01-04,NaN,00:10,Thursday,123 PARKWAY FOREST DRI,Utilized Off Route,0,0,NaN,3400
659,2024-01-05,NaN,05:43,Friday,MOUNT DENNIS DIVISION,Operations - Operator,0,0,NaN,0


In [13]:
# Now, we will see how many for NaN routes have delays
na_route[na_route['Min Delay'] == 0].count()

Date         363
Route          0
Time         363
Day          363
Location     363
Incident     363
Min Delay    363
Min Gap      363
Direction     68
Vehicle      363
dtype: int64

Since all the NaN routes have 0 delays, it is safe to drop these rows. We will also remove all other null values to ensure a clean dataset to work with

In [14]:
ttc_clean = ttc_clean.dropna()
ttc_clean.isna().sum()

Date         0
Route        0
Time         0
Day          0
Location     0
Incident     0
Min Delay    0
Min Gap      0
dtype: int64

<br>

___
## Visualizing

Now it's the fun part: visualizations!